In [1]:
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

batch_size = 2

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


In [4]:
from utils.pspnet import PSPNet

net = PSPNet(n_classes=150)

state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

n_classes = 21
net.decode_feature.classification = nn.Conv2d(
    in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

net.aux.classification = nn.Conv2d(
    in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  
            nn.init.constant_(m.bias, 0.0)


net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)

Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))

In [5]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

In [6]:
class PSPLoss(nn.Module):
    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight 

    def forward(self, outputs, targets):
        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

        return loss+self.aux_weight*loss_aux


criterion = PSPLoss(aux_weight=0.4)


In [7]:
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)


In [8]:
def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    net.to(device)

    torch.backends.cudnn.benchmark = True

    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    iteration = 1
    logs = []

    # multiple minibatch
    batch_multiplier = 3

    for epoch in range(num_epochs):

        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0 
        epoch_val_loss = 0.0 

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() 
                scheduler.step() 
                optimizer.zero_grad()
                print('（train）')

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()  
                    print('-------------')
                    print('（val）')
                else:
                   
                    continue

            count = 0  # multiple minibatch
            for imges, anno_class_imges in dataloaders_dict[phase]:
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                
                if (phase == 'train') and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(imges)
                    loss = criterion(
                        outputs, anno_class_imges.long()) / batch_multiplier

                    if phase == 'train':
                        loss.backward()  
                        count -= 1  # multiple minibatch

                        if (iteration % 10 == 0):  
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    else:
                        epoch_val_loss += loss.item() * batch_multiplier

        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss /
                     num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

    torch.save(net.state_dict(), 'weights/pspnet50_' +
               str(epoch+1) + '.pth')


In [10]:
num_epochs = 10
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)


cuda:0
-------------
Epoch 1/10
-------------
（train）
イテレーション 10 || Loss: 0.6948 || 10iter: 4.4443 sec.
イテレーション 20 || Loss: 0.9359 || 10iter: 4.3896 sec.
イテレーション 30 || Loss: 0.7485 || 10iter: 4.3678 sec.
イテレーション 40 || Loss: 0.3689 || 10iter: 4.3875 sec.
イテレーション 50 || Loss: 0.6341 || 10iter: 4.4141 sec.
イテレーション 60 || Loss: 1.2326 || 10iter: 4.4320 sec.
イテレーション 70 || Loss: 0.7648 || 10iter: 4.4270 sec.
イテレーション 80 || Loss: 2.1241 || 10iter: 4.4097 sec.
イテレーション 90 || Loss: 0.7412 || 10iter: 4.4047 sec.
イテレーション 100 || Loss: 0.7777 || 10iter: 4.4201 sec.
イテレーション 110 || Loss: 0.6610 || 10iter: 4.4061 sec.
イテレーション 120 || Loss: 0.3256 || 10iter: 4.4099 sec.
イテレーション 130 || Loss: 0.5868 || 10iter: 4.4124 sec.
イテレーション 140 || Loss: 0.6622 || 10iter: 4.4034 sec.
イテレーション 150 || Loss: 0.5399 || 10iter: 4.3953 sec.
イテレーション 160 || Loss: 1.0238 || 10iter: 4.4094 sec.
イテレーション 170 || Loss: 1.5876 || 10iter: 4.4161 sec.
イテレーション 180 || Loss: 1.0056 || 10iter: 4.3985 sec.
イテレーション 190 || Loss: 1.8141 || 10iter

イテレーション 1550 || Loss: 0.5839 || 10iter: 4.5886 sec.
イテレーション 1560 || Loss: 0.1639 || 10iter: 4.6853 sec.
イテレーション 1570 || Loss: 0.1787 || 10iter: 4.8270 sec.
イテレーション 1580 || Loss: 0.1371 || 10iter: 4.6466 sec.
イテレーション 1590 || Loss: 0.3802 || 10iter: 4.4294 sec.
イテレーション 1600 || Loss: 0.2099 || 10iter: 4.4337 sec.
イテレーション 1610 || Loss: 0.3987 || 10iter: 4.4281 sec.
イテレーション 1620 || Loss: 0.5409 || 10iter: 4.4227 sec.
イテレーション 1630 || Loss: 0.1997 || 10iter: 4.4678 sec.
イテレーション 1640 || Loss: 0.2297 || 10iter: 4.4619 sec.
イテレーション 1650 || Loss: 0.3531 || 10iter: 4.5183 sec.
イテレーション 1660 || Loss: 0.5787 || 10iter: 4.4523 sec.
イテレーション 1670 || Loss: 0.1198 || 10iter: 4.4345 sec.
イテレーション 1680 || Loss: 0.5051 || 10iter: 4.3693 sec.
イテレーション 1690 || Loss: 0.4206 || 10iter: 4.3698 sec.
イテレーション 1700 || Loss: 0.2060 || 10iter: 4.5560 sec.
イテレーション 1710 || Loss: 0.4199 || 10iter: 4.8593 sec.
イテレーション 1720 || Loss: 0.6132 || 10iter: 4.4760 sec.
イテレーション 1730 || Loss: 0.3721 || 10iter: 4.6602 sec.
イテレーション 1740

イテレーション 3080 || Loss: 0.0716 || 10iter: 4.4733 sec.
イテレーション 3090 || Loss: 0.1096 || 10iter: 4.7482 sec.
イテレーション 3100 || Loss: 0.3142 || 10iter: 4.9526 sec.
イテレーション 3110 || Loss: 0.2835 || 10iter: 4.9331 sec.
イテレーション 3120 || Loss: 0.0475 || 10iter: 4.8389 sec.
イテレーション 3130 || Loss: 0.1662 || 10iter: 4.7421 sec.
イテレーション 3140 || Loss: 0.1982 || 10iter: 4.6679 sec.
イテレーション 3150 || Loss: 0.0477 || 10iter: 4.5491 sec.
イテレーション 3160 || Loss: 0.1411 || 10iter: 4.5116 sec.
イテレーション 3170 || Loss: 0.1201 || 10iter: 4.9884 sec.
イテレーション 3180 || Loss: 0.1363 || 10iter: 4.4951 sec.
イテレーション 3190 || Loss: 0.1348 || 10iter: 4.7632 sec.
イテレーション 3200 || Loss: 0.3434 || 10iter: 4.7782 sec.
イテレーション 3210 || Loss: 0.0806 || 10iter: 4.7595 sec.
イテレーション 3220 || Loss: 0.0807 || 10iter: 4.9220 sec.
イテレーション 3230 || Loss: 0.0486 || 10iter: 4.6077 sec.
イテレーション 3240 || Loss: 0.2553 || 10iter: 4.4889 sec.
イテレーション 3250 || Loss: 0.1463 || 10iter: 4.5164 sec.
イテレーション 3260 || Loss: 0.2423 || 10iter: 4.5235 sec.
イテレーション 3270

イテレーション 4600 || Loss: 0.2000 || 10iter: 4.5443 sec.
イテレーション 4610 || Loss: 0.3828 || 10iter: 4.4555 sec.
イテレーション 4620 || Loss: 0.2563 || 10iter: 4.4374 sec.
イテレーション 4630 || Loss: 0.1371 || 10iter: 4.4287 sec.
イテレーション 4640 || Loss: 0.1559 || 10iter: 4.4152 sec.
イテレーション 4650 || Loss: 0.2454 || 10iter: 4.5476 sec.
イテレーション 4660 || Loss: 0.2435 || 10iter: 4.4613 sec.
イテレーション 4670 || Loss: 0.1341 || 10iter: 4.4628 sec.
イテレーション 4680 || Loss: 0.4345 || 10iter: 4.4175 sec.
イテレーション 4690 || Loss: 0.1245 || 10iter: 4.7475 sec.
イテレーション 4700 || Loss: 0.1939 || 10iter: 4.6605 sec.
イテレーション 4710 || Loss: 0.1539 || 10iter: 4.5614 sec.
イテレーション 4720 || Loss: 0.1455 || 10iter: 4.5555 sec.
イテレーション 4730 || Loss: 0.0974 || 10iter: 4.4801 sec.
イテレーション 4740 || Loss: 0.2430 || 10iter: 4.4672 sec.
イテレーション 4750 || Loss: 0.1010 || 10iter: 4.5228 sec.
イテレーション 4760 || Loss: 0.2322 || 10iter: 4.8028 sec.
イテレーション 4770 || Loss: 0.5388 || 10iter: 4.8771 sec.
イテレーション 4780 || Loss: 0.2587 || 10iter: 4.5162 sec.
イテレーション 4790

イテレーション 6130 || Loss: 0.1819 || 10iter: 4.4153 sec.
イテレーション 6140 || Loss: 0.0835 || 10iter: 4.4013 sec.
イテレーション 6150 || Loss: 0.3021 || 10iter: 4.3880 sec.
イテレーション 6160 || Loss: 0.2175 || 10iter: 4.4093 sec.
イテレーション 6170 || Loss: 0.1171 || 10iter: 4.3882 sec.
イテレーション 6180 || Loss: 0.2212 || 10iter: 4.4155 sec.
イテレーション 6190 || Loss: 0.3209 || 10iter: 4.4378 sec.
イテレーション 6200 || Loss: 0.0742 || 10iter: 4.4278 sec.
イテレーション 6210 || Loss: 0.0353 || 10iter: 4.4123 sec.
イテレーション 6220 || Loss: 0.2231 || 10iter: 4.3871 sec.
イテレーション 6230 || Loss: 0.1280 || 10iter: 4.4098 sec.
イテレーション 6240 || Loss: 1.7717 || 10iter: 4.4073 sec.
イテレーション 6250 || Loss: 0.0757 || 10iter: 4.3895 sec.
イテレーション 6260 || Loss: 0.1733 || 10iter: 4.3915 sec.
イテレーション 6270 || Loss: 0.9794 || 10iter: 4.4025 sec.
イテレーション 6280 || Loss: 0.2921 || 10iter: 4.3962 sec.
イテレーション 6290 || Loss: 0.1226 || 10iter: 4.3931 sec.
イテレーション 6300 || Loss: 0.3200 || 10iter: 4.4095 sec.
イテレーション 6310 || Loss: 0.4412 || 10iter: 4.3944 sec.
イテレーション 6320